<a href="https://colab.research.google.com/github/PUNEETSUBHANJI/NYC_Taxi_Trip_prediction/blob/main/NYC_Taxi_Trip_Time_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project  : NYC Taxi trip duration Prediction :: Predicting total ride duration of taxi trips in NYC using the given Dataset

Problem Description : 

Task is to build a model that predicts the total ride duration of taxi trips in New York City. primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and many other variables.


## Description of Data

The dataset is based on the 2016 NYC Cab trip record data, The data was originally published by the NYC Taxi and Limousine Commission.
The data was sampled and cleaned for the performing this project.

Based on individual trip attributes, you should predict the duration of each trip in the test set.

<b> NYC Taxi Data.csv </b> - The Dataset contains 1458644 trip records


### Details of Dataset
* #### id - A unique id for each trip.
* #### vendor_id - An identification code for the provider associated to the trip record.
* #### pickup_datetime - Time when the meter started or the date and time of pickup.
* #### dropoff_datetime -  Time and date that the metre was turned off or dropped off.
* #### passenger_count - Number of passengers as reported by the driver.
* #### pickup_longitude - Longitude details of Pick up or when the meter started.
* #### pickup_latitude -  latitude details of Pick up or when the meter started.
* #### dropoff_longitude - Longitude details of dropoff or when the meter turned off.
* #### dropoff_latitude - latitude details of dropoff or when the meter turned of
* #### store_and_fwd_flag - This flag indicates us, whether the trip record was stored in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip.
* #### trip_duration - Its our target variable and it is duration of the trip in seconds.


# Mounting of Drive, Loading Data and Importing of the required libraries


In [1]:
# Mounting of google drive to fetch the Raw_DATA_SET
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

# Importing necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split,GridSearchCV
import statsmodels.formula.api as sm
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import datetime as dt
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

In [4]:
#Fetching the given dataet using pandas
taxi_data = pd.read_csv('/content/drive/MyDrive/Projects- Almabetter_Puneet/NYC Taxi Time prediction/NYC Taxi Data.csv')